## Frequency Table - code for only data which has cases

In [ ]:
import pandas as pd
# Assuming the uploaded file is named 'data.xlsx'
file_path = 'Data_test_16May2024.xlsx'

# Read the Excel file into a DataFrame
data1 = pd.read_excel(file_path)

# Display the first few rows of the DataFrame
print(data1.head())

In [ ]:
import pandas as pd                    # Frequncy table - model-1 with total with all the variables excel download
import openpyxl
from openpyxl.styles import Alignment
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, NamedStyle

# Replace actual missing values with 'Missing'
data1.fillna('Missing', inplace=True)
# Assuming 'fixed_column' contains the name of the fixed column variable
fixed_column = 'Case_Control_new'

# Create a new Excel workbook
workbook = openpyxl.Workbook()

# Remove the default sheet created by openpyxl (Sheet)
workbook.remove(workbook.active)

sheet = workbook.create_sheet(title=f'Crosstab_{fixed_column}vs{column}')
sheet.merge_cells('A1:A2')
sheet.merge_cells('B1:B2')
sheet.merge_cells('C1:C2')
sheet.merge_cells('D1:D2')
#sheet.merge_cells('E1:E2')
# Write headings
#sheet.append(['Cross-Tabulation', f'{fixed_column} vs {column}'])
#sheet.append([])  # Add an empty row for spacing
thin = Side(border_style="thin", color="000000")
thick = Side(border_style="thick", color="000000")
thin_border = Border(top=Side(style='thin'), bottom=Side(style='thin'))  # Thick border at top and bottom
thin_header_border = Border(top=Side(style='thick'), bottom=Side(style='thick'))
thick_border = Border(bottom=Side(style='thick'))
for row in sheet['A1:D2']:
    for cell in row:
      #print(cell)
      cell.font = Font(bold=True)
      cell.alignment  = Alignment(horizontal='center', vertical='center')
      cell.border = Border(top=thick, left=thick, right=thick, bottom=thick)
sheetRow = 3
# Dictionary to store results
results = {}

# Function to adjust percentages to sum to 100
def adjust_percentages(df):
    for col in df.columns:
        total_percent = df[col].sum()
        if total_percent != 100:
            max_idx = df[col].idxmax()
            df.at[max_idx, col] += 100 - total_percent
    return df

# Get list of all categorical variables excluding the fixed column
categorical_columns = data1.select_dtypes(include='object').columns.tolist()
categorical_columns.remove(fixed_column)

# Iterate over each categorical variable in the dataset
for i, column in enumerate(categorical_columns):
    # Create crosstab with frequencies
    crosstab_freq = pd.crosstab(data1[column], data1[fixed_column], margins=False)
    crosstab_perc = pd.crosstab(data1[column], data1[fixed_column], normalize='columns') * 100

    # Round percentages carefully to ensure they sum to 100
    crosstab_perc = crosstab_perc.applymap(lambda x: int(x) if x < 0.5 else round(x))

    # Adjust percentages to sum to 100
    crosstab_perc = adjust_percentages(crosstab_perc)

    # Concatenate required columns: 'Cases' and 'Case_percent'
    result_df = pd.concat([
        crosstab_freq.iloc[:, 0],  # Cases
        crosstab_perc.iloc[:, 0]],  # Case percent
        axis=1, keys=['Cases', 'Case_percent'])

    # Check if 'Missing' category exists and add it with 0 values if needed
    if 'Missing' not in result_df.index and 'Missing' in data1[column].unique():
        result_df.loc['Missing'] = [0, 0]

    # Only add a row for 'Total' if it's the last variable
    if i == len(categorical_columns) - 1:
        total_cases = result_df['Cases'].sum()
        result_df.loc['Total'] = [total_cases, 100]

    # Reorder rows: main categories first, then 'Missing', then 'Total'
    main_categories = result_df.index.difference(['Missing', 'Total']).tolist()
    if 'Missing' in result_df.index and 'Total' in result_df.index:
        result_df = result_df.loc[main_categories + ['Missing', 'Total']]
    elif 'Missing' in result_df.index:
        result_df = result_df.loc[main_categories + ['Missing']]
    elif 'Total' in result_df.index:
        result_df = result_df.loc[main_categories + ['Total']]
    else:
        result_df = result_df.loc[main_categories]

    # Store results in the dictionary
    results[f'Crosstab_{fixed_column}_vs_{column}'] = result_df

    # Print the crosstab results
    print(f"\nResults for {column} vs {fixed_column}:")
    print(result_df)

    # Create a new sheet for each cross-tabulation
    sheet.cell(row=1, column=1, value='Variables')
    sheet.cell(row=1, column=2, value='Category')
    sheet.cell(row=1, column=3, value='Cases (n)')
    sheet.cell(row=1, column=4, value='Case_percent')
    #sheet.cell(row=1, column=5, value='Attack rate (%)')
    # Write the structured table (including headings) to the sheet
    sheet.cell(row=sheetRow, column=1, value=column).border = thick_border
    sheet.merge_cells('A'+str(sheetRow)+':A'+str(sheetRow+(len(result_df) -1)))
    start_cell = sheet.cell(row=sheetRow, column=1)
    end_cell = sheet.cell(row=sheetRow+(len(result_df) -1), column=1)
    sheet.merge_cells(start_row=start_cell.row, start_column=start_cell.column,
                      end_row=end_cell.row, end_column=end_cell.column)
    merged_cell = sheet.cell(row=start_cell.row, column=start_cell.column)
    merged_cell.alignment = Alignment(horizontal='center', vertical='center')
    merged_cell.font = Font(bold=True)
    merged_cell.border = thin_header_border
    irow = 1
    for r_idx, row in enumerate(result_df.iterrows(), start=sheetRow):  # Start from row 3 (after headings)
        sheet.cell(row=r_idx, column=2, value=row[0])
        for c_idx, value in enumerate(row[1], start=3):
            sheet.cell(row=r_idx, column=c_idx, value=value)
            if irow == len(result_df):
              #print(value)
              last_row = sheet.cell(row=r_idx, column=c_idx)
              last_row.border=thick_border
        if irow == len(result_df):
            #print(r_idx)
            last_row = sheet.cell(row=r_idx, column=2)
            last_row.border=thick_border
        irow = irow + 1
        sheetRow = sheetRow + 1
# Save the workbook
workbook.save("New_table.xlsx")
from google.colab import files
files.download('New_table.xlsx')